In [ ]:
import re
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import numpy as np
from PIL import Image


def split_and_savelabel(result, index, path, selected_img,filename):
    ori_img = selected_img

    x1, y1, x2, y2 = result.xyxy[0]  # 경계 상자 좌표

    conf = result.conf[0]  # 정확도 가져오기
    cls = result.cls[0]  # 클래스 이름 가져오기
    label = loaded_result.names[int(cls)]  # 클래스 이름을 숫자로 변환

    print(0, float(x1), float(y1), float(x2), float(y2))

    print(type(ori_img))

    # BGR에서 RGB로 변환
    ori_img_rgb = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    numpy_to_PIL = Image.fromarray(ori_img_rgb)  # Numpy array to PIL image
    train_img_path = os.path.join(path, 'train/images', f'{filename}.jpg')

    numpy_to_PIL.save(train_img_path, 'JPEG')  # save PIL image


def make_folder():
    # img와 label을 저장할 폴더
    img_label = "img_label_folder"

    # img_label에 넣을 train, test, val 폴더
    subfolders = ["train", "test", "val"]

    # train, test, val 폴더 안에 넣을 서브 폴더들
    sub_subfolders = ["images", "labels"]

    # img_label_folder 폴더 만들기
    if not os.path.exists(img_label):
        os.makedirs(img_label)

    # train, test, val 폴더 만들기
    for subfolder in subfolders:
        path = os.path.join(img_label, subfolder)
        if not os.path.exists(path):
            os.makedirs(path)
        for subsub in sub_subfolders:
            sub_path = os.path.join(path, subsub)
            if not os.path.exists(sub_path):
                os.makedirs(sub_path)
            print(sub_path)

    return img_label


# label의 배경과 글자 크기 설정 메서드
def make_box(result, count, original_img):
    x1, y1, x2, y2 = result.xyxy[0]  # 경계 상자 좌표
    conf = result.conf[0]
    cls = result.cls[0]
    label = loaded_result.names[int(cls)]

    # 경계 상자 그리기
    cv2.rectangle(original_img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # 텍스트 위치 계산 및 조정
    text = f'object: {count}, acc: {conf:.2f}'
    font_scale = 2.0  # 텍스트 크기 조정
    font_thickness = 4  # 텍스트 두께 조정
    text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    text_w, text_h = text_size
    text_x = int(x1)
    text_y = int(y1) - 10

    if text_y < text_h:
        text_y = int(y1) + text_h + 10

    # 텍스트 배경 그리기
    cv2.rectangle(original_img, (text_x, text_y - text_h), (text_x + text_w, text_y), (255, 0, 0), -1)
    # 텍스트 그리기
    cv2.putText(original_img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)


# 폴더 train, test, val 폴더 만들고 path는 img_label에 저장
img_label = make_folder()

# 저장된 pickle 데이터를 가져오기 위해 파일명 가져오기
results_folder = "result_pickle_notebook"

# 0, 1, 10, 100 ... 과 같이 정렬되지 않고 0, 1, 2, 3, 4... 순으로 정렬되게 하기 위해서
sorted_result = sorted(os.listdir(results_folder), key=lambda s: int(re.search(r'\d+', s).group()))

# 저장된 모든 Results 객체 불러오기 및 시각화
for filename in sorted_result:
    file_path = os.path.join(results_folder, filename)
    with open(file_path, 'rb') as f:
        loaded_result = pickle.load(f)

    count = 0
    original_img = loaded_result.orig_img.copy()  # 원본 이미지 복사

    for box in loaded_result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # 경계 상자 좌표
        conf = box.conf[0]
        cls = box.cls[0]
        label = loaded_result.names[int(cls)]

        if label == 'person':
            count += 1
            make_box(box, count, original_img)

    # 그림을 초기화하고 이미지 표시
    plt.figure()
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.show()

    select_object = int(input("Select object number (0 to continue): "))
    if select_object == 0:
        continue
    else:
        x1, y1, x2, y2 = loaded_result.boxes[select_object - 1].xyxy[0]
        selected_img = loaded_result.orig_img.copy()
        selected_ori_img = loaded_result.orig_img.copy()

        make_box(loaded_result.boxes[select_object - 1], select_object, selected_img)

        # 이 부분에 dataframe 만들고 split 하기
        split_and_savelabel(loaded_result.boxes[select_object - 1], select_object, img_label, selected_ori_img, filename)

        # 그림을 초기화하고 이미지 표시
        plt.figure()
        plt.imshow(cv2.cvtColor(selected_img, cv2.COLOR_BGR2RGB))
        plt.show()
        print(type(selected_img))

In [ ]:
import re
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import numpy as np
from PIL import Image


def split_and_savelabel(result, index, path, selected_img,filename):
    ori_img = selected_img

    x1, y1, x2, y2 = result.xyxy[0]  # 경계 상자 좌표
    
    img_idx=int(re.search(r'\d+', filename ).group())

    conf = result.conf[0]  # 정확도 가져오기
    cls = result.cls[0]  # 클래스 이름 가져오기
    label = loaded_result.names[int(cls)]  # 클래스 이름을 숫자로 변환

    print(0, float(x1), float(y1), float(x2), float(y2))

    print(type(ori_img))

    # BGR에서 RGB로 변환
    ori_img_rgb = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    numpy_to_PIL = Image.fromarray(ori_img_rgb)  # Numpy array to PIL image
    train_img_path = os.path.join(path, 'train/images', f'{img_idx}.jpg')

    numpy_to_PIL.save(train_img_path, 'JPEG')  # save PIL image

    train_label_path = os.path.join(path,'train/labels',f'{img_idx}.text')
    # 만약 파일이 없다면 만들고 쓸 파일을 open 하기
    f=open(train_label_path,"a+")
    f.write(f"0 {x1} {y1} {x2} {y2}")


    
def make_folder():
    # img와 label을 저장할 폴더
    img_label = "img_label_folder"

    # img_label에 넣을 train, test, val 폴더
    subfolders = ["train", "test", "val"]

    # train, test, val 폴더 안에 넣을 서브 폴더들
    sub_subfolders = ["images", "labels"]

    # img_label_folder 폴더 만들기
    if not os.path.exists(img_label):
        os.makedirs(img_label)

    # train, test, val 폴더 만들기
    for subfolder in subfolders:
        path = os.path.join(img_label, subfolder)
        if not os.path.exists(path):
            os.makedirs(path)
        for subsub in sub_subfolders:
            sub_path = os.path.join(path,subsub)
            if not os.path.exists(sub_path):
                os.makedirs(sub_path)
            print(sub_path)
        
    return img_label




# label의 배경과 글자 크기 설정 메서드
def make_box(result,count,original_img):
    x1, y1, x2, y2 = result.xyxy[0]  # 경계 상자 좌표
    conf = result.conf[0]
    cls = result.cls[0]
    label = loaded_result.names[int(cls)]

    # 경계 상자 그리기
    cv2.rectangle(original_img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    
    # 텍스트 위치 계산 및 조정
    text = f'object : {count}, acc : {conf:.2f}'
    font_scale = 2.0  # 텍스트 크기 조정
    font_thickness = 4  # 텍스트 두께 조정
    text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    text_w, text_h = text_size
    text_x = int(x1)
    text_y = int(y1) - 10
    
    if text_y < text_h:
        text_y = int(y1) + text_h + 10
    
    # 텍스트 배경 그리기
    cv2.rectangle(original_img, (text_x, text_y - text_h), (text_x + text_w, text_y), (255, 0, 0), -1)
    # 텍스트 그리기
    cv2.putText(original_img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)



# 폴더 train, test, val 폴더 만들고 path는 img_label에 저장
img_label = make_folder()

# 저장된 pickle 데이터를 가져오기 위해 파일명 가져오기
results_folder = "result_pickle_notebook"

# 0, 1, 10, 100 ... 과 같이 정렬되지 않고 0, 1, 2, 3, 4... 순으로 정렬되게 하기 위해서
sorted_result = sorted(os.listdir(results_folder), key=lambda s: int(re.search(r'\d+', s).group()))

# 저장된 모든 Results 객체 불러오기 및 시각화
for filename in sorted_result:
    file_path = os.path.join(results_folder, filename)
    with open(file_path, 'rb') as f:
        loaded_result = pickle.load(f)
    
    count = 0
    original_img = loaded_result.orig_img.copy()  # 원본 이미지 복사

    for box in loaded_result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # 경계 상자 좌표
        conf = box.conf[0]
        cls = box.cls[0]
        label = loaded_result.names[int(cls)]

        if label == 'person':
            count += 1
            make_box(box,count,original_img)


    # 그림을 초기화하고 이미지 표시
    plt.figure()
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.show()

    select_object = int(input("Select object number (0 to continue): "))
    if select_object == 0:
        continue
    else:
        x1, y1, x2, y2 = loaded_result.boxes[select_object-1].xyxy[0]
        selected_img = loaded_result.orig_img.copy()
        selected_ori_img = loaded_result.orig_img.copy()

        make_box(loaded_result.boxes[select_object-1],select_object,selected_img)
        
        # 이 부분에 dataframe 만들고 split 하기
        split_and_savelabel(loaded_result.boxes[select_object-1],select_object,img_label,selected_ori_img,filename)


        # 그림을 초기화하고 이미지 표시
        plt.figure()
        plt.imshow(cv2.cvtColor(selected_img, cv2.COLOR_BGR2RGB))
        plt.show()
        print(type(selected_img))

